In [ ]:
# AI Hub에서 다운로드한 상품 데이터셋을 images/labels 폴더로 이동 -> 각각 하나의 폴더에 데이터를 모으기 위함


import os
import shutil


def move_files(source_folder, destination_folder):
    # 대상 폴더가 있는지 확인
    os.makedirs(destination_folder, exist_ok=True)

    # source_folder의 각 하위 폴더들에 대해 모두 수행
    for root, _, files in os.walk(source_folder):
        for file in files:
            source_file_path = os.path.join(root, file)
            destination_file_path = os.path.join(destination_folder, file)

            # 파일을 destination_folder 위치로 이동
            shutil.move(source_file_path, destination_file_path)

if __name__ == "__main__":
    source_folder = "C:/Users/nogo0/Downloads/datasets/yolo_test/train/images"
    destination_folder = "C:/Users/nogo0/Downloads/datasets/test/train/images"
    move_files(source_folder, destination_folder)


In [ ]:
# labels 폴더에서 불필요한 meta.xml 파일 삭제 -> yolov8 학습에 불필요하기 때문


import glob
import os


[os.remove(f) for f in glob.glob('C:/Users/nogo0/Downloads/datasets/yolo_test/valid/labels/*meta*')]

In [ ]:
# 각 상품 데이터 폴더별 파일 개수 세는 코드 -> 지금까지 전처리한 파일의 개수가 맞는지 중간 점검 용도


import os


def count_files_in_subfolders(folder_path):
    # 폴더 이름 및 해당 폴더 파일 수를 저장하는 딕셔너리
    folder_file_count = {}

    # 폴더 및 하위 폴더마다 확인
    for root, dirs, files in os.walk(folder_path):
        # 현재 폴더의 파일 개수 계산
        file_count = len(files)

        # 폴더 이름 및 해당 폴더 파일 수를 딕셔너리에 저장
        folder_name = os.path.basename(root)
        folder_file_count[folder_name] = file_count

    return folder_file_count

if __name__ == "__main__":
    # 대상 폴더 경로
    folder_path = "C:/Users/nogo0/Downloads/datasets/yolo_test/train/images"

    # 폴더 이름 및 해당 폴더 파일 수 가져오기
    result = count_files_in_subfolders(folder_path)

    # 결과 출력
    for folder, count in result.items():
        print(f"Folder '{folder}' has {count} files.")


In [ ]:
# yolov8 학습용 데이터를 구분하기 위해 따로 다른 폴더로 이동


import os
import shutil


def move_top_files(source_folder, destination_folder, num_files=10000):
    # 대상 폴더가 있는지 확인
    os.makedirs(destination_folder, exist_ok=True)

    # source_folder의 모든 파일 목록을 가져온 후 정렬
    files = sorted(os.listdir(source_folder))

    # num_files개의 파일을 대상 폴더로 이동
    for file in files[:num_files]:
        source_file_path = os.path.join(source_folder, file)
        destination_file_path = os.path.join(destination_folder, file)

        # 파일을 destination_folder로 이동
        shutil.move(source_file_path, destination_file_path)

if __name__ == "__main__":
    source_folder = "C:/Users/nogo0/Downloads/datasets/test/train/images"
    destination_folder = "C:/Users/nogo0/Downloads/datasets/test2/train/images"
    move_top_files(source_folder, destination_folder, num_files=11740)


In [ ]:
# yaml 파일 생성시 한글 인코딩 깨지는 거 막기 위한 코드


import os


os.environ['PYTHONIOENCODING'] = 'utf-8'

In [3]:
# yolov8 학습을 위한 yaml 파일 자동 생성 -> 이후에 nc 개수 맞는지 확인 후 잘못됐으면 names 개수 세서 수정


import os
import yaml
import re


def generate_yaml_from_xml(xml_folder, yaml_path):
    class_set = set()
    nc_value = None

    # xml_folder의 xml 파일에 대해 반복
    for root, _, files in os.walk(xml_folder):
        for xml_file in files:
            if xml_file.endswith('.xml'):
                xml_file_path = os.path.join(root, xml_file)

                # xml 파일 분석
                with open(xml_file_path, 'r', encoding='utf-8') as file:
                    content = file.read()

                # <name> 태그에서 클래스명 추출
                class_name_match = re.search(r'<name>(.*?)</name>', content)
                if class_name_match:
                    class_name = class_name_match.group(1)
                    class_set.add(class_name)

                # <filename> 태그에서 정수 클래스 추출
                filename_match = re.search(r'<filename>(\d+)_', content)
                if filename_match:
                    nc_value = int(filename_match.group(1))

    # 일관된 순서를 위해 리스트로 변환
    classes = list(class_set)

    # yaml_content 생성
    yaml_content = {'names': classes, 'nc': nc_value}

    with open(yaml_path, 'w', encoding='utf-8-sig') as yaml_file:
        yaml.dump(yaml_content, yaml_file, default_flow_style=False, allow_unicode=True)


if __name__ == "__main__":
    xml_folder = "C:/Users/nogo0/Downloads/[라벨]과자4"  # 라벨링된 xml 파일들이 있는 폴더 경로
    yaml_path = "C:/Users/nogo0/Downloads/datasets/last_yolo_test/data.yaml"  # 생성될 yaml 파일 경로
    generate_yaml_from_xml(xml_folder, yaml_path)


In [5]:
# 다운받은 상품 데이터 labels의 xml 라벨링 파일을 yolo 학습에 맞게 txt 파일로 변환


import os
import re
import yaml


def generate_yolo_labels(xml_folder, yaml_path, output_folder):
    # yaml 파일 불러오기
    with open(yaml_path, 'r', encoding='utf-8-sig') as yaml_file:
        yaml_data = yaml.safe_load(yaml_file)

    # 클래스명 추출 및 매핑 생성
    class_mapping = {class_name: str(idx) for idx, class_name in enumerate(yaml_data['names'])}

    # xml_folder의 각 xml 파일마다 처리
    for root, _, files in os.walk(xml_folder):
        for xml_file in files:
            if xml_file.endswith('.xml'):
                xml_file_path = os.path.join(root, xml_file)

                # xml 파일 분석
                with open(xml_file_path, 'r', encoding='utf-8-sig') as file:
                    content = file.read()

                # <name> 태그에서 클래스명 추출
                class_name_match = re.search(r'<name>(.*?)</name>', content)
                if class_name_match:
                    class_name = class_name_match.group(1)

                    # 클래스명이 yaml 파일에 있는지 확인
                    if class_name in class_mapping:
                        class_id = class_mapping[class_name]

                        # 바운딩 박스 좌표 추출
                        bbox_match = re.search(r'<bndbox>(.*?)</bndbox>', content, re.DOTALL)
                        if bbox_match:
                            bbox_content = bbox_match.group(1)
                            xmin = re.search(r'<xmin>(\d+)</xmin>', bbox_content).group(1)
                            ymin = re.search(r'<ymin>(\d+)</ymin>', bbox_content).group(1)
                            xmax = re.search(r'<xmax>(\d+)</xmax>', bbox_content).group(1)
                            ymax = re.search(r'<ymax>(\d+)</ymax>', bbox_content).group(1)

                            # YOLO 좌표 계산
                            width = int(xmax) - int(xmin)
                            height = int(ymax) - int(ymin)
                            x_center = (int(xmin) + width / 2) / 2988  # 이미지 너비
                            y_center = (int(ymin) + height / 2) / 2988  # 이미지 높이
                            box_width = width / 2988
                            box_height = height / 2988

                            # YOLO 학습 포맷
                            yolo_line = f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}"

                            # YOLO txt 파일 저장
                            txt_file_path = os.path.join(output_folder, f"{os.path.splitext(xml_file)[0]}.txt")
                            with open(txt_file_path, 'w') as txt_file:
                                txt_file.write(yolo_line)

if __name__ == "__main__":
    xml_folder = "C:/Users/nogo0/Downloads/last_yolo_test_valid_labels"  # 라벨링된 xml 파일들이 있는 폴더 경로
    yaml_path = "C:/Users/nogo0/Downloads/datasets/last_yolo_test/data.yaml"  # yaml 파일 경로
    output_folder = "C:/Users/nogo0/Downloads/datasets/last_yolo_test/valid/labels"  # YOLO 라벨 파일을 저장할 폴더 경로
    generate_yolo_labels(xml_folder, yaml_path, output_folder)


In [17]:
# xml 파일을 txt로 변환하는데 몇 개는 생성이 안 되는 문제 발생
# 그래서 xml, txt 파일을 한 폴더에 모아두고 아래 코드 돌려서 txt 파일 없는 xml 파일에 수동으로 txt 파일 생성


import os


# 대상 폴더의 모든 파일 목록 가져오기
file_list = os.listdir("C:/Users/nogo0/Downloads/datasets/test/train/labels")

# 파일 이름만 추출
file_name_list = []
for file_name in file_list:
    file_name_list.append(file_name.split(".")[0])

# 파일 이름의 빈도수 계산
file_name_count_dict = {}
for file_name in file_name_list:
    if file_name in file_name_count_dict:
        file_name_count_dict[file_name] += 1
    else:
        file_name_count_dict[file_name] = 1

# 고유한 파일 이름 찾기
unique_file_names = []
for file_name, count in file_name_count_dict.items():
    if count == 1:
        unique_file_names.append(file_name)

# 결과 출력
for unique_file_name in unique_file_names:
    print(unique_file_name)

20160_00_m_7
20160_00_s_19
20222_00_m_19
20222_00_m_7
20222_00_s_20
30068_30_s_1
30100_00_s_7
30220_30_m_7
30220_30_s_19
30227_00_s_7
35432_60_m_8


In [ ]:
# txt 파일 하나도 빠짐없이 다 만든 이후에 아래 코드 실행해서 xml 파일을 다시 원래 폴더로 옮김


import os
import shutil


def move_xml_files(source_path, destination_path):
    # 대상 폴더가 없다면 생성
    if not os.path.exists(destination_path):
        os.makedirs(destination_path)
    
    # 소스 폴더에서 .xml 파일 찾기
    xml_files = [f for f in os.listdir(source_path) if f.endswith('.xml')]
    
    # .xml 파일들을 대상 폴더로 이동
    for xml_file in xml_files:
        source_file_path = os.path.join(source_path, xml_file)
        destination_file_path = os.path.join(destination_path, xml_file)
        shutil.move(source_file_path, destination_file_path)
        print(f"Moved: {xml_file} to {destination_path}")


source_path = "C:/Users/nogo0/Downloads/datasets/test/train/labels"  # 현재 폴더 경로
destination_path = "C:/Users/nogo0/Downloads/datasets/test_train_labels"  # 이동할 대상 폴더 경로
move_xml_files(source_path, destination_path)


In [ ]:
# 아래 코드 활용하여 images에는 있는데 labels에는 없는 파일 이름 찾아서 적절히 수동으로 조정해주기

import os

# 첫 번째 대상 폴더의 모든 파일 목록 가져오기
file_list_a = os.listdir("C:/Users/nogo0/Downloads/datasets/test/valid/images")

# 두 번째 대상 폴더의 모든 파일 목록 가져오기
file_list_b = os.listdir("C:/Users/nogo0/Downloads/datasets/test/valid/labels")

# 파일 이름만 추출
file_name_list_a = []
for file_name in file_list_a:
    file_name_list_a.append(file_name.split(".")[0])

file_name_list_b = []
for file_name in file_list_b:
    file_name_list_b.append(file_name.split(".")[0])

# 첫 번째 폴더에만 있는 파일 이름 찾기
unique_file_names = []
for file_name in file_name_list_a:
    if file_name not in file_name_list_b:
        unique_file_names.append(file_name)

# 결과 출력
for unique_file_name in unique_file_names:
    print(unique_file_name)


In [3]:
# YOLOv8 학습 진행
# 1만개 데이터 기준 한 에폭 도는데 약 2분 30초 정도 걸림

from ultralytics import YOLO


model = YOLO('yolov8s.pt')
model.train(data='C:/Users/nogo0/Downloads/datasets/kt_fire_ver4/data.yaml',
            epochs=20,
            patience=5,
            batch=4,
            seed=42
            )


100%|██████████| 21.5M/21.5M [00:01<00:00, 11.8MB/s]


New https://pypi.org/project/ultralytics/8.0.237 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.231 🚀 Python-3.9.12 torch-2.0.1 CUDA:0 (NVIDIA GeForce RTX 3080, 10239MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:/Users/nogo0/Downloads/datasets/kt_fire_ver4/data.yaml, epochs=20, time=None, patience=5, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train42, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None

train: Scanning C:\Users\nogo0\Downloads\datasets\kt_fire_ver4\train\labels.cache... 10155 images, 75 backgrounds, 0 corrupt: 100%|██████████| 10155/10155 [00:00<?, ?it/s]
val: Scanning C:\Users\nogo0\Downloads\datasets\kt_fire_ver4\valid\labels.cache... 509 images, 19 backgrounds, 0 corrupt: 100%|██████████| 509/509 [00:00<?, ?it/s]


Plotting labels to runs\detect\train42\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)


2024/01/08 15:07:32 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2024/01/08 15:07:32 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.


...

                   all        509        549      0.967      0.952      0.985      0.774



20 epochs completed in 0.746 hours.
Optimizer stripped from runs\detect\train42\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train42\weights\best.pt, 22.5MB

Validating runs\detect\train42\weights\best.pt...
Ultralytics YOLOv8.0.231 🚀 Python-3.9.12 torch-2.0.1 CUDA:0 (NVIDIA GeForce RTX 3080, 10239MiB)
Model summary (fused): 168 layers, 11128680 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 64/64 [00:03<00:00, 18.58it/s]


                   all        509        549      0.967      0.952      0.985      0.774
                 bYott        509         65       0.98      0.985      0.994      0.831
            honeyAmond        509         78       0.96      0.949      0.986      0.749
              hotBreak        509         73      0.909      0.817      0.954      0.711
                 ohYes        509         73          1      0.999      0.995       0.86
             onionRing        509         91      0.901        0.9      0.964      0.717
              ozingZip        509         39          1      0.963      0.994      0.786
              pokaChip        509         63      0.994          1      0.995      0.754
           shrimpGGang        509         67      0.994          1      0.995      0.785
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\train42
MLflow: results logged to runs\mlflow
MLflow: disable with 'yolo settings mlflow

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000208B0762340>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047